<h1><center>Laboratorio 3: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastian Calderon Altamirano
- Nombre de alumno 2: Camila Vera Gallardo


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/...../)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/MyDrive/MDS7202/online_retail_data.pickle'
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Se define el path del archivo
path = '/content/drive/MyDrive/MDS7202/online_retail_data.pickle'

# Se carga el archivo pickle en un DataFrame
df_retail = pd.read_pickle(path)

# Verificar las primeras filas
df_retail.head()

### 1. Función para explorar características [0.5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [0.3 puntos]
- Imprima un conteo de datos nulos por variable [0.2 puntos]

**Nota**: Para generar los gráficos no es obligatorio el uso de `plotly`, pero si es altamente recomendado. Pueden encontrar más información de esta librería en este [enlace](https://plotly.com/python/).

**Respuesta:**

In [ ]:
def explore_data(dataframe_in):
    # Hace una copia del dataframe
    df = dataframe_in.copy()

    # Conteo de nulos por variable
    nulos = df.isna().sum().sort_values(ascending=False)
    print(" Conteo de datos nulos por variable")
    print(nulos)

    # Histogramas
    import plotly.express as px

    fig1 = px.histogram(
            df, x="Price",
            range_x=[0, 100],
            title="Histograma de Price")
    fig1.update_traces(xbins=dict(start=1, end=100, size=2))
    fig1.update_layout(bargap=0.05)
    fig1.show()

    fig2 = px.histogram(
            df, x="Quantity",
            range_x=[0, 250],
            title="Histograma de Quantity")
    fig2.update_traces(xbins=dict(start=0, end=250, size=10))
    fig2.update_layout(bargap=0.05)
    fig2.show()

explore_data(df_retail)

### 2. Eliminando outliers [1.0 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [0.5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

**Hint:** tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).

**Nota:** No modificar el método set_output de la clase IQR

**Respuesta:**

In [ ]:
from typing import Iterable, Optional, Union

class IQR(BaseEstimator, TransformerMixin):

  def __init__(self,
               lam: float = 1.5,
               columns: Optional[Union[Iterable[str], str]] = None,
               copy: bool = True):

    self.lam = lam                    # Lambda
    self.columns = columns  # Columnas
    self.copy = copy                # Copia del DataFrame

  def fit(self, X, y=None):
    X_df = X.to_frame() if isinstance(X, pd.Series) else X

    # Determinar columnas objetivo
    cols = [self.columns] if isinstance(self.columns, str) else list(self.columns)

    # Guardar columnas y quartiles
    self.columns_ = cols
    q1 = X_df[self.columns_].quantile(0.25, interpolation="linear")
    q3 = X_df[self.columns_].quantile(0.75, interpolation="linear")
    iqr = q3 - q1

    self.q1_ = q1
    self.q3_ = q3
    self.iqr_ = iqr
    self.lower_ = q1 - self.lam * iqr
    self.upper_ = q3 + self.lam * iqr

    return self

  def transform(self, X):
    is_series = isinstance(X, pd.Series)
    X_df = X.to_frame() if is_series else X
    if self.copy:
      X_df = X_df.copy()

    for col in self.columns_:
      lower = self.lower_[col]
      upper = self.upper_[col]
      mask = (X_df[col] < lower) | (X_df[col] > upper)
      X_df.loc[mask, col] = np.nan

    return X_df.iloc[:, 0] if is_series else X_df

  def set_output(self, transform='default'):
    # No modificar esta función
    return self

#### 2.2 Creación del Pipeline [0.5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [0.1 puntos]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [0.1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [0.1 puntos]


**Hint:** El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.

**Nota:** Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasaran por cada pipeline
numerical_columns = ["Price", "Quantity"]
categorical_columns = [c for c in df_retail.columns if c not in numerical_columns]

# Definicion del pipeline
numeric_transformations = Pipeline(steps=[
    ("iqr", IQR(lam=1.5, columns=numerical_columns, copy=True))
])

# ColumnTransformer
column_transformer = ColumnTransformer([
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', 'passthrough', categorical_columns)
], verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

# Graficos
print("\n--- Exploración ANTES de IQR (df_retail) ---")
explore_data(df_retail)

print("\n--- Exploración DESPUÉS de IQR (df_iqr) ---")
explore_data(df_iqr)

*Reporte los cambios observados aquí.*

### 3. Agregando un imputer al pipeline [1.0 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [0.1 puntos]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando |un dataframe llamado `df_mean_imputer`. [0.1 puntos]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [0.1 puntos]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [0.2 puntos]

**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer

numerical_columns = ["Price", "Quantity"]
categorical_columns = [c for c in df_retail.columns if c not in numerical_columns]

# Pipeline para categóricas
categoric_transformations = Pipeline(steps=[
    ("mode_imputer", SimpleImputer(strategy="most_frequent"))
])

# Pipeline numérico para mean imputer
numeric_transformations_mean = Pipeline(steps=[
    ("iqr", IQR(lam=1.5, columns=numerical_columns, copy=True)),
    ("mean_imputer", SimpleImputer(strategy="mean"))
])

# ColumnTransformer para mean imputer
column_transformer_mean = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformations_mean, numerical_columns),
        ('categorical', categoric_transformations, categorical_columns)
    ],
    verbose_feature_names_out=False,
    n_jobs=-1
)
column_transformer_mean.set_output(transform='pandas')
df_mean_imputer = column_transformer_mean.fit_transform(df_retail)

# Comparacion de distribuciones
print("\n--- Exploración DESPUÉS de IQR (df_iqr) ---")
explore_data(df_iqr)

print("\n--- Exploración TRAS IQR + MEAN IMPUTER (df_mean_imputer) ---")
explore_data(df_mean_imputer)

In [ ]:
# Se aplica IGR primero
df_iqr_applied = df_retail.copy()
iqr_transformer = IQR(lam=1.5, columns=numerical_columns, copy=True)
df_iqr_applied = iqr_transformer.fit_transform(df_iqr_applied)

# Se identifica solo las filas con valores nulos en columnas numericas
null_mask = df_iqr_applied[numerical_columns].isnull().any(axis=1)
rows_with_nulls = df_iqr_applied[null_mask]
rows_without_nulls = df_iqr_applied[~null_mask]

print(f"Filas con nulos: {len(rows_with_nulls)}/{len(df_iqr_applied)}")

if len(rows_with_nulls) > 0:
    # Se aplica KNN solo a las filas con valores nulos
    knn_imputer = KNNImputer(n_neighbors=3, weights="uniform")

    # Imputar solo las columnas numericas
    numerical_data = rows_with_nulls[numerical_columns]
    imputed_numerical = knn_imputer.fit_transform(numerical_data)

    # Reconstruir el dataframe
    rows_with_nulls_imputed = rows_with_nulls.copy()
    rows_with_nulls_imputed[numerical_columns] = imputed_numerical

    # Combinar con las filas sin nulos
    df_knn_imputer = pd.concat([rows_without_nulls, rows_with_nulls_imputed])
else:
    df_knn_imputer = df_iqr_applied

explore_data(df_knn_imputer)

4. **Comparación entre `df_mean_imputer` y `df_iqr`**: El método IQR dejó varios valores nulos en variables como `Price`, `Quantity`, `InvoiceDate` y `Country`, mientras que el Mean Imputer completó todos los valores nulos, lo que asegura que el conjunto de datos esté completamente imputado y listo para el análisis, haciendo que el Mean Imputer sea más efectivo.

66. **Comparación entre `df_knn_imputer` y `df_iqr`**: Ambos métodos dejaron valores nulos en columnas como `InvoiceDate` y `Country`, pero IQR eliminó algunos outliers, lo que permitió que varias columnas se completaran, mientras que KNN Imputer no logró imputar estos valores correctamente, indicando que ambos métodos no fueron 100% efectivos en imputar todos los valores faltantes.

7. **Comparación entre `df_knn_imputer` y `df_mean_imputer`**: El Mean Imputer es el mejor método, ya que imputó correctamente todos los valores nulos, mientras que el KNN Imputer dejó valores faltantes en columnas clave como `InvoiceDate` y `Country`. Por lo tanto, Mean Imputer es el mejor para asegurar un conjunto de datos completo y sin valores faltantes.

### 4. Creación de nuevas features [2.0 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [1.0 puntos]

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
def custom_features(dataframe_in):
    # Hacemos una copia para no modificar el original
    df = dataframe_in.copy()

    # Calculamos el gasto total por transaccion
    df['Total'] = df['Price'] * df['Quantity']

    # Encontramos la fecha mas reciente en el dataset (hoy)
    hoy = df['Date'].max()

    # Agrupamos por Customer ID
    grouped = df.groupby('Customer ID')

    # Calculamos Length: diferencia en dias entre primera y ultima visita
    length = (grouped['Date'].max() - grouped['Date'].min()).dt.days

    # Calculamos Recency: dias desde la ultima compra hasta hoy
    recency = (hoy - grouped['Date'].max()).dt.days

    # Calculamos Frequency: numero de visitas/transacciones
    frequency = grouped.size()

    # Calculamos Monetary: gasto total promedio por visita
    monetary = grouped['Total'].sum() / frequency

    # Calculamos Periodicity: desviacion estandar de los intervalos entre visitas (IVT)
    # Primero ordenamos las fechas por cliente
    df_sorted = df.sort_values(['Customer ID', 'Date'])
    # Calculamos diferencias consecutivas por grupo
    ivt = df_sorted.groupby('Customer ID')['Date'].diff().dt.days
    # Calculamos la desviacion estandar de los IVT por cliente
    periodicity = df_sorted.groupby('Customer ID')['Date'].apply(
        lambda x: x.diff().dt.days.std() if len(x) > 1 else 0
    )

    # Juntamos todos los resultados en un DataFrame
    lrmfp = pd.DataFrame({
        'Length': length,
        'Recency': recency,
        'Frequency': frequency,
        'Monetary': monetary,
        'Periodicity': periodicity
    }).reset_index()

    return lrmfp

#### 4.2 Agregando las custom features [1.0 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [0.1 puntos]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [0.1 puntos]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [0.5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [0.3 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import numpy as np
import pandas as pd

# Definimos las columnas numéricas y categóricas
numerical_columns = ["Price", "Quantity"]
categorical_columns = [c for c in df_retail.columns if c not in numerical_columns]

# Pipeline para variables categóricas
categoric_transformations = Pipeline(steps=[
    ("mode_imputer", SimpleImputer(strategy="most_frequent"))
])


numeric_transformations_optimized = Pipeline(steps=[
    ("iqr", IQR(lam=1.5, columns=numerical_columns, copy=True)),
    ("mean_imputer", SimpleImputer(strategy="mean"))
])

# ColumnTransformer optimizado
column_transformer_optimized = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformations_optimized, numerical_columns),
        ('categorical', categoric_transformations, categorical_columns)
    ],
    verbose_feature_names_out=False
)

# Establecer que la salida será un DataFrame
column_transformer_optimized.set_output(transform='pandas')

# Función optimizada para las nuevas características LRMFP
def custom_features_optimized(dataframe_in):
    """
    Función optimizada para calcular características LRMFP
    """
    df = dataframe_in.copy()

    # Asegurar que tenemos Customer ID válidos
    df = df.dropna(subset=['Customer ID'])

    # Convertir la fecha y manejar errores
    if 'InvoiceDate' in df.columns:
        df['Date'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
    elif 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    else:
        raise ValueError("No se encontró columna de fecha")


    valid_dates = df['Date'].notna()
    if not valid_dates.any():
        print("Advertencia: No hay fechas válidas en el dataset")
        return pd.DataFrame(columns=['Customer ID', 'Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity'])

    # Trabajar solo con fechas válidas para cálculos LRMFP
    df_valid = df[valid_dates].copy()

    # Calcular el total por transacción
    df_valid['Total'] = df_valid['Price'] * df_valid['Quantity']

    # Fecha más reciente del dataset
    hoy = df_valid['Date'].max()


    # Cálculos agrupados optimizados
    agg_dict = {
        'Date': ['min', 'max', 'count'],
        'Total': 'sum'
    }

    grouped = df_valid.groupby('Customer ID').agg(agg_dict)


    grouped.columns = ['FirstDate', 'LastDate', 'Frequency', 'TotalSum']
    grouped = grouped.reset_index()

    # Cálculos básicos
    grouped['Length'] = (grouped['LastDate'] - grouped['FirstDate']).dt.days
    grouped['Recency'] = (hoy - grouped['LastDate']).dt.days
    grouped['Monetary'] = grouped['TotalSum'] / grouped['Frequency']

    # Periodicity optimizada
    df_sorted = df_valid[['Customer ID', 'Date']].sort_values(['Customer ID', 'Date'])

    # Calcular diferencias de tiempo de forma vectorizada
    df_sorted['DateDiff'] = df_sorted.groupby('Customer ID')['Date'].diff().dt.days

    # Calcular desviación estándar de los intervalos
    periodicity_dict = {}
    for customer_id in df_sorted['Customer ID'].unique():
        customer_diffs = df_sorted[df_sorted['Customer ID'] == customer_id]['DateDiff'].dropna()
        if len(customer_diffs) > 0:
            periodicity_dict[customer_id] = customer_diffs.std()
        else:
            periodicity_dict[customer_id] = 0

    # Mapear periodicity
    grouped['Periodicity'] = grouped['Customer ID'].map(periodicity_dict).fillna(0)

    # Redondear para mejor presentación
    grouped['Monetary'] = grouped['Monetary'].round(2)
    grouped['Periodicity'] = grouped['Periodicity'].round(1)

    result = grouped[['Customer ID', 'Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']]

    print(f"Características LRMFP calculadas para {len(result)} clientes")
    return result

# Definir el pipeline completo optimizado
retail_pipeline = Pipeline([
    ('col_transformer', column_transformer_optimized),
    ('custom_features', FunctionTransformer(custom_features_optimized, validate=False))
])

# Aplicar el pipeline al DataFrame
df_custom = retail_pipeline.fit_transform(df_retail)

print(f"DataFrame resultante: {df_custom.shape}")
print("\nPrimeras 5 filas:")
print(df_custom.head())

# Función optimizada para explorar las nuevas características LRMFP
def explore_data_custom_optimized(dataframe_in):
    df = dataframe_in.copy()

    # Conteo de nulos por variable
    nulos = df.isna().sum().sort_values(ascending=False)
    print("Conteo de datos nulos por variable")
    print(nulos)

    # Estadísticas descriptivas
    print("\nEstadísticas descriptivas:")
    print(df.describe())

    # Histogramas para cada variable LRMFP
    lrmfp_cols = ['Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']

    import plotly.express as px

    for col in lrmfp_cols:
        if col in df.columns:
            # Eliminar outliers extremos para mejor visualización
            q1 = df[col].quantile(0.01)
            q99 = df[col].quantile(0.99)
            df_plot = df[(df[col] >= q1) & (df[col] <= q99)]

            fig = px.histogram(
                df_plot,
                x=col,
                title=f'Histograma de {col}',
                nbins=50
            )
            fig.show()

# Explorar las nuevas características
explore_data_custom_optimized(df_custom)

### 5. MinMax Scaler [1.0 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [0.5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Si X es un DataFrame, calculamos el mínimo y el máximo para cada columna numérica
        if isinstance(X, pd.DataFrame):
            self.min_ = X.min()
            self.max_ = X.max()
        else:
            raise ValueError("El input X debe ser un DataFrame.")

        return self

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            # Normalizamos con la fórmula Min-Max
            X_normalized = (X - self.min_) / (self.max_ - self.min_)
        else:
            raise ValueError("El input X debe ser un DataFrame.")

        return X_normalized

    def set_output(self, transform='default'):
        # No modificar este método
        return self


#### 5.2 Incorporando MinMax al pipeline [0.5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [0.1 puntos]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables.  [0.2 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
# Primero, definimos el pipeline para las transformaciones numéricas, incluyendo MinMax
numeric_transformations_minmax = Pipeline(steps=[
    ("iqr", IQR(lam=1.5, columns=numerical_columns, copy=True)),  # Usar el IQR como antes
    ("minmax", MinMax())  # Agregar el transformador MinMax al pipeline
])

# Actualizamos el ColumnTransformer para aplicar las transformaciones numéricas y categóricas
column_transformer_minmax = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformations_minmax, numerical_columns),
        ('categorical', categoric_transformations, categorical_columns)
    ],
    verbose_feature_names_out=False  # Fijamos esto en False
)

# Establecemos que la salida será un DataFrame
column_transformer_minmax.set_output(transform='pandas')

# Aplicamos el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems
df_minmax = column_transformer_minmax.fit_transform(df_retail)

# Función para explorar las distribuciones de las variables
def explore_data(dataframe_in):
    df = dataframe_in.copy()

    # Conteo de nulos por variable
    nulos = df.isna().sum().sort_values(ascending=False)
    print("Conteo de datos nulos por variable")
    print(nulos)

    # Histogramas para las variables Price y Quantity (en este caso se mostrarán todas las variables numéricas)
    import plotly.express as px

    fig1 = px.histogram(
            df, x="Price",
            range_x=[0, 100],
            title="Histograma de Price")
    fig1.update_traces(xbins=dict(start=1, end=100, size=2))
    fig1.update_layout(bargap=0.05)
    fig1.show()

    fig2 = px.histogram(
            df, x="Quantity",
            range_x=[0, 250],
            title="Histograma de Quantity")
    fig2.update_traces(xbins=dict(start=0, end=250, size=10))
    fig2.update_layout(bargap=0.05)
    fig2.show()

# Exploramos las distribuciones en los datos originales y transformados
explore_data(df_retail)

explore_data(df_minmax)


### 6. Pregunta teórica [0.5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Es importante usar pipelines en el Feature Engineering porque ayuda a organizar y automatizar el proceso de preparación de datos de manera clara y consistente. Esto facilita el trabajo con los datos al asegurarse de que las transformaciones se apliquen de la misma manera cada vez, evitando errores comunes como la fuga de información. Además, los pipelines hacen que el proceso sea más fácil de entender y reproducir, lo que mejora la capacidad para tomar decisiones informadas y trabajar en equipo.